In [13]:
# Load packages and symbols
library(tidyverse)
library(lubridate)
library(readxl)
library(highcharter)
library(tidyquant)
library(timetk)
library(tibbletime)
library(quantmod) 
library(PerformanceAnalytics)
library(scales)
library(sandwich) # new!
library(SharpeR) # new!
library(PortfolioAnalytics) # new!
symbols = c("SPY","EFA","IJS","EEM","AGG")

In [14]:
prices <-
  getSymbols(symbols, src = 'yahoo',
             from = "2012-12-31",
             to = "2022-01-17",
             auto.assign = TRUE,
             warnings = FALSE) %>%
  map(~Ad(get(.))) %>%
  reduce(merge) %>%
  `colnames<-`(symbols)
prices_monthly =
  to.monthly(prices, indexAt="lastof", OHLC = FALSE)
returns_xts <-
  Return.calculate(prices_monthly,
                 method = "log") %>%
na.omit()

In [15]:
w <- c(0.25,
       0.25,
       0.20,
       0.20,
       0.10)

tibble(w,symbols)

asset1 <- returns_xts[,1]
asset2 <- returns_xts[,2]
asset3 <- returns_xts[,3]
asset4 <- returns_xts[,4]
asset5 <- returns_xts[,5]
portfolio_returns_byhand <-
  (w[1] * asset1) +
  (w[2] * asset2) +
  (w[3] * asset3) +
  (w[4] * asset4) +
  (w[5] * asset5)
names(portfolio_returns_byhand) <- "returns"


portfolio_returns_byhand <-
  t(w%*%t(returns_xts))
names(portfolio_returns_byhand) <- "returns"


portfolio_returns_xts_rebalanced_monthly <-
  Return.portfolio(returns_xts, weights = w,
                   rebalance_on = "months") %>%
  `colnames<-`("returns")
head(portfolio_returns_xts_rebalanced_monthly,3)

w,symbols
<dbl>,<chr>
0.25,SPY
0.25,EFA
0.20,IJS
0.20,EEM
0.10,AGG


                 returns
2013-01-31  0.0308488138
2013-02-28 -0.0008696856
2013-03-31  0.0186624693

In [16]:
portfolio_returns_tq_rebalanced_monthly <-
  returns_long %>%
  tq_portfolio(assets_col  = asset,
               returns_col = returns,
               weights     = w,
               col_rename  = "returns",
               rebalance_on = "months")

ERROR: Error in eval(expr, envir, enclos): object 'returns_long' not found
